In [2]:
import pandas as pd
import openpyxl

GwanGwang = pd.read_excel('jejudoGwanGwang.xlsx')
jejudoMatJip = pd.read_excel('jejudoMatJip.xlsx')
jejuMatJip = pd.read_excel('jejuMatJip.xlsx')
jejuYeoHang = pd.read_excel('jejuYeoHang.xlsx')

In [3]:
# 중복데이터는 거르고 결합..하고 싶다
# df1.combine_first(df2) ====> df1을 기준으로 df2를 중복없이 결합하겠다.
jejuGwanGwang1 = GwanGwang.combine_first(jejudoMatJip)
jejuGwanGwang1 = jejuGwanGwang1.combine_first(jejuMatJip)
jejuGwanGwang1 = jejuGwanGwang1.combine_first(jejuYeoHang)

In [4]:
jejuGwanGwang1['place'].value_counts()

Jeju                    243
Jeju-do                 126
Jungle Book by Alice    109
고래배꼽                     87
제주도 크리스마스 박물관            59
                       ... 
GD Cafe                   1
제주좋은날                     1
Guzasien -구좌지앵            1
도두항                       1
동탄2에 산다                   1
Name: place, Length: 784, dtype: int64

In [21]:
import requests

def place_find(s) :
    # 접속 url
    url = 'http://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(s)
    
    # headers 입력
    headers = {
        # 카카오지도 접속
        "Authorization" : "KakaoAK 095562b7356dfd5360e2c6860f69d4b2"
    }
    # API 요청
    palces = requests.get(url, headers=headers).json()['documents']
    try :
        palces = palces[0]
        jejuGwanGwang1_name  = palces['place_name']
        위도 = palces['y']
        경도 = palces['x']
        
        result = [jejuGwanGwang1_name, 위도, 경도]
        
        return palces
    except:
        pass

In [22]:
# 테스트
place_find('청초수물회&섭국 [신관]')

KeyError: 'documents'

In [23]:
# 혹시 있을지 모르는 결측치를 제거하고싶다
a = jejuGwanGwang1['place'].dropna()
a.value_counts()

Jeju                    243
Jeju-do                 126
Jungle Book by Alice    109
고래배꼽                     87
제주도 크리스마스 박물관            59
                       ... 
GD Cafe                   1
제주좋은날                     1
Guzasien -구좌지앵            1
도두항                       1
동탄2에 산다                   1
Name: place, Length: 784, dtype: int64

In [24]:
# 위경도 가져오기
jeju_palce_list = []

for a in a :
    try :
        # 카카오 API사용한 사용자 함수로 던지기
        data = place_find(a)
        jeju_palce_list.append(data)
    except :
        pass

In [12]:
# 복사
jeju_palce_list_1 = jeju_palce_list
# None 값 삭제
jeju_palce_list_1 =  [x for x in jeju_palce_list_1 if x is not None]
# 확인
jeju_palce_list_1

[['서귀포잠수함', '33.2393033784206', '126.558616052674'],
 ['청초수물회 속초본점', '38.19214711524039', '128.5905399503119'],
 ['엄지네포장마차 속초점', '38.203049336827476', '128.5917760658347'],
 ['명월국민학교', '33.3887377492719', '126.264871685447'],
 ['파리바게뜨 연산로타리점', '35.1870429046049', '129.082691569532'],
 ['제주에인감귤밭', '33.25656069063887', '126.5390139270361'],
 ['제주에인감귤밭', '33.25656069063887', '126.5390139270361'],
 ['서귀포잠수함', '33.2393033784206', '126.558616052674'],
 ['새별오름', '33.366277591139294', '126.3577306657398'],
 ['성이시돌목장', '33.3477698762559', '126.327321076199'],
 ['월령선인장군락지', '33.3778792539349', '126.2154316324'],
 ['서울맛집', '37.5383079971898', '127.054025642442'],
 ['토끼썸', '33.52192638345875', '126.89881498224804'],
 ['CENACULUM JEJU', '33.4449549374343', '126.30129764229179'],
 ['북청형제 원조국수집', '37.4235896024628', '126.693893441784'],
 ['제주도', '33.379777816446165', '126.54587355630036'],
 ['한라산1100고지', '33.3580781709788', '126.462219691112'],
 ['제주레포츠랜드', '33.48206819907232', '126.6387551242765'],


In [13]:
jeju_palce = pd.DataFrame(jeju_palce_list_1)
# 컬럼 명 지정해주기
jeju_palce.columns = ['place_name', '위도', '경도']
# '제주도'는 의미 없을거 같아서 제거
jeju_palce = jeju_palce[(jeju_palce['place_name'] != '제주도')]
jeju_palce

,place_name,위도,경도
0,서귀포잠수함,33.2393033784206,126.558616052674
1,청초수물회 속초본점,38.19214711524039,128.5905399503119
2,엄지네포장마차 속초점,38.203049336827476,128.5917760658347
3,명월국민학교,33.3887377492719,126.264871685447
4,파리바게뜨 연산로타리점,35.1870429046049,129.082691569532
...,...,...,...
1862,아도록,33.35880368535034,126.20175404761244
1863,고집돌우럭 중문점,33.2579811121134,126.416704762779
1864,소노벨 제주,33.541376642880266,126.6716678612834
1865,연돈,33.258895288625645,126.40715814631936


In [14]:
# 제주도 지도
import folium
jejuMap = folium.Map(location=[33.36, 126.52], zoom_start = 10)

In [15]:
from folium.plugins import MarkerCluster

In [16]:
import folium

from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(jejuMap)

for i in range(len(jeju_palce)):
    folium.Marker(
        location = [jeju_palce.iloc[i]['위도'], jeju_palce.iloc[i]['경도']],
        popup = jeju_palce.iloc[i]['place_name'],
        icon = folium.Icon(color='blue',icon='ok'),
    ).add_to(marker_cluster)

jejuMap